In [1]:
import pickle
import os
import regex as re
import tiktoken

In [2]:
with open('cleaned_data.pkl', 'rb') as file:
	list_of_strings = pickle.load(file)

In [3]:
print(len(list_of_strings))
print(list_of_strings[0])
print(len(list_of_strings[0]))

2732634
Once upon a time, there was a little girl named Sue. Sue was very adventurous. She loved to play outside and explore. One day, Sue saw a big box in her room. She did not know what was inside.
Sue's mommy said, "Sue, sit down and open the box." Sue sat down and opened the box. Inside, she found a beautiful uniform. It was red and blue with gold stars. Sue put on the uniform and felt very special.
Sue wore her uniform outside to play. She met her friend Tom. Tom said, "Wow, Sue! Your uniform is so pretty!" Sue smiled and said, "Thank you, Tom. Let's go on an adventure!" Together, they explored the park and had a great day. Sue knew that her special uniform made her feel even more adventurous.
699


In [4]:
# turn it into one string instead of a list of strings
combined_string = ''.join(list_of_strings)

# find the unique characters
chars = sorted(list(set(combined_string)))
v = len(chars)
print('\n', chars, v)
del combined_string


 ['\n', ' ', '!', '"', '$', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] 74


In [5]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
char_encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers


In [43]:
vocab_size = 128 # the desired final vocabulary size
num_merges = vocab_size - v

In [44]:
base_indices = char_encode(chars)
print(base_indices)
origin = [ "symbol" if i < 22 else "letter" for i in base_indices]  # Track token origin
print(origin)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73]
['symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'symbol', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'letter', 'l

In [45]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): # Pythonic way to iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  # in the list of ints (ids), replace all consecutive occurences of pair with the new token idx
  newids = []
  i = 0
  while i < len(ids):
    # if we are not at the very last position AND the pair matches, replace it
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

In [46]:
# turning a usable subset of the data into one string
combined_string = ''.join(list_of_strings[:1000])
tokens = char_encode(combined_string)
ids = list(tokens) # copy so we don't destroy the original list

# now let's actually do it
merges = {} # (int, int) -> int
for i in range(num_merges):
    #print(i)
    stats = get_stats(ids)

    # Modified pair selection logic:
    while True:
        pair = max(stats, key=stats.get)  # Get the most frequent pair initially
        #print(pair)

        if origin[pair[0]] != origin[pair[1]]: # Check if origins differ
            #print(origin[pair[0]],origin[pair[1]],origin[pair[0]] != origin[pair[1]])
            del stats[pair]
        else:  # If no valid pairs left, break out of the loop
            #print(origin[pair[0]],origin[pair[1]],origin[pair[0]] != origin[pair[1]])
            break
    else:
        break  # Valid pair found 
    
    pair = max(stats, key=stats.get)
    #print(pair)

    idx = v + i
    print(f"merging {pair} into a new token {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx
    origin.append(origin[pair[0]])

merging (55, 52) into a new token 74
merging (8, 1) into a new token 75
merging (48, 61) into a new token 76
merging (67, 74) into a new token 77
merging (52, 51) into a new token 78
merging (6, 1) into a new token 79
merging (67, 62) into a new token 80
merging (76, 51) into a new token 81
merging (56, 61) into a new token 82
merging (65, 52) into a new token 83
merging (62, 68) into a new token 84
merging (70, 48) into a new token 85
merging (56, 67) into a new token 86
merging (55, 48) into a new token 87
merging (41, 74) into a new token 88
merging (52, 65) into a new token 89
merging (85, 66) into a new token 90
merging (48, 72) into a new token 91
merging (52, 61) into a new token 92
merging (62, 60) into a new token 93
merging (56, 66) into a new token 94
merging (62, 61) into a new token 95
merging (56, 60) into a new token 96
merging (48, 65) into a new token 97
merging (66, 48) into a new token 98
merging (56, 51) into a new token 99
merging (59, 52) into a new token 100
merg

In [47]:
print("tokens length:", len(tokens)) # remember tokens are our original tokens
print("ids length:", len(ids)) # and ids are new tokens we've made
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 804737
ids length: 578026
compression ratio: 1.39X


In [48]:
# Ensure the tokenizers directory exists
if not os.path.exists('./tokenizers'):
    os.makedirs('./tokenizers')

# Prepare the tokenizer data to be saved
tokenizer_data = {
    'stoi': stoi,  # Character to integer mapping
    'merges': merges  # Merges dictionary
}

# Save the tokenizer data using pickle
with open(f'./tokenizers/tiny_stories_tokenizer_{vocab_size}.model', 'wb') as f:
    pickle.dump(tokenizer_data, f)

print("Tokenizer saved successfully.")

Tokenizer saved successfully.


In [49]:
# Load the tokenizer data using pickle
with open(f'./tokenizers/tiny_stories_tokenizer_{vocab_size}.model', 'rb') as f:
    loaded_tokenizer_data = pickle.load(f)

# Extract the stoi mapping and merges from the loaded data
loaded_stoi = loaded_tokenizer_data['stoi']
loaded_merges = loaded_tokenizer_data['merges']

print("Tokenizer loaded successfully.")

Tokenizer loaded successfully.


In [50]:
print(stoi)
print(merges)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '$': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ':': 19, ';': 20, '?': 21, 'A': 22, 'B': 23, 'C': 24, 'D': 25, 'E': 26, 'F': 27, 'G': 28, 'H': 29, 'I': 30, 'J': 31, 'K': 32, 'L': 33, 'M': 34, 'N': 35, 'O': 36, 'P': 37, 'Q': 38, 'R': 39, 'S': 40, 'T': 41, 'U': 42, 'V': 43, 'W': 44, 'X': 45, 'Y': 46, 'Z': 47, 'a': 48, 'b': 49, 'c': 50, 'd': 51, 'e': 52, 'f': 53, 'g': 54, 'h': 55, 'i': 56, 'j': 57, 'k': 58, 'l': 59, 'm': 60, 'n': 61, 'o': 62, 'p': 63, 'q': 64, 'r': 65, 's': 66, 't': 67, 'u': 68, 'v': 69, 'w': 70, 'x': 71, 'y': 72, 'z': 73}
{(55, 52): 74, (8, 1): 75, (48, 61): 76, (67, 74): 77, (52, 51): 78, (6, 1): 79, (67, 62): 80, (76, 51): 81, (56, 61): 82, (65, 52): 83, (62, 68): 84, (70, 48): 85, (56, 67): 86, (55, 48): 87, (41, 74): 88, (52, 65): 89, (85, 66): 90, (48, 72): 91, (52, 61): 92, (62, 60): 93, (56, 66): 94, (62, 61): 95, (56, 60): 96, (48, 65): 97, (66, 48):

In [51]:
class SimpleTokenizer:
    def __init__(self, stoi, merges):
        self.stoi = stoi
        self.merges = merges
        self.itos = {i: s for s, i in stoi.items()}  # Inverse mapping for decoding

        self.vocab_len = len(stoi) + len(merges)

    def encode(self, text):
        # Convert the text to a list of token IDs, using space for unknown characters
        tokens = [self.stoi.get(c, self.stoi[' ']) for c in text]

        # Perform merging with the possibility of nested merges
        i = 0
        while i < len(tokens) - 1:
            pair = (tokens[i], tokens[i + 1])
            if pair in self.merges:
                # Replace the current pair with its merged token
                merged_token = self.merges[pair]
                tokens[i] = merged_token
                del tokens[i + 1]

                # Move back to handle possible nested merges
                if i > 0:
                    i -= 1
            else:
                i += 1

        return tokens

    def decode(self, tokens):
        def expand_token(token):
            # Base case: if the token is a direct mapping, return its character
            if token in self.itos:
                return self.itos[token]
            # Recursive case: if the token is a merged token, expand its constituents
            elif token in self.merges.values():
                pair = next(key for key, value in self.merges.items() if value == token)
                return ''.join(expand_token(t) for t in pair)
            # Fallback for unknown tokens
            else:
                return ''

        # Decode each token in the list, handling nested merges recursively
        return ''.join(expand_token(token) for token in tokens)

# Example usage
# Assuming loaded_stoi and loaded_merges are already loaded from the tokenizer.model file

tokenizer = SimpleTokenizer(loaded_stoi, loaded_merges)

# Encoding text
encoded_text = tokenizer.encode("JULIET:\nO Romeo, Romeo! wherefore art thou R")
print("Encoded:", encoded_text)

# Decoding back
decoded_text = tokenizer.decode(encoded_text)
print("Decoded:", decoded_text)

Encoded: [31, 42, 33, 30, 26, 41, 19, 0, 36, 1, 39, 93, 52, 62, 79, 39, 93, 52, 62, 2, 1, 70, 126, 52, 53, 104, 52, 1, 97, 67, 1, 67, 55, 84, 1, 39]
Decoded: JULIET:
O Romeo, Romeo! wherefore art thou R


In [52]:
for i in range(vocab_size):
    print(f"{i}: '{tokenizer.decode([i])}'")

0: '
'
1: ' '
2: '!'
3: '"'
4: '$'
5: '''
6: ','
7: '-'
8: '.'
9: '0'
10: '1'
11: '2'
12: '3'
13: '4'
14: '5'
15: '6'
16: '7'
17: '8'
18: '9'
19: ':'
20: ';'
21: '?'
22: 'A'
23: 'B'
24: 'C'
25: 'D'
26: 'E'
27: 'F'
28: 'G'
29: 'H'
30: 'I'
31: 'J'
32: 'K'
33: 'L'
34: 'M'
35: 'N'
36: 'O'
37: 'P'
38: 'Q'
39: 'R'
40: 'S'
41: 'T'
42: 'U'
43: 'V'
44: 'W'
45: 'X'
46: 'Y'
47: 'Z'
48: 'a'
49: 'b'
50: 'c'
51: 'd'
52: 'e'
53: 'f'
54: 'g'
55: 'h'
56: 'i'
57: 'j'
58: 'k'
59: 'l'
60: 'm'
61: 'n'
62: 'o'
63: 'p'
64: 'q'
65: 'r'
66: 's'
67: 't'
68: 'u'
69: 'v'
70: 'w'
71: 'x'
72: 'y'
73: 'z'
74: 'he'
75: '. '
76: 'an'
77: 'the'
78: 'ed'
79: ', '
80: 'to'
81: 'and'
82: 'in'
83: 're'
84: 'ou'
85: 'wa'
86: 'it'
87: 'ha'
88: 'The'
89: 'er'
90: 'was'
91: 'ay'
92: 'en'
93: 'om'
94: 'is'
95: 'on'
96: 'im'
97: 'ar'
98: 'sa'
99: 'id'
100: 'le'
101: 'll'
102: 'ing'
103: 'ot'
104: 'or'
105: 'st'
106: 'ir'
107: '.
'
108: 'am'
109: 'ne'
110: 'They'
111: 'pl'
112: 'at'
113: 'il'
114: 'lo'
115: 'ig'
116: 'He'
117: 'v